## Gender Classification with Perceptron using First Names

In [390]:
# import all the required libraries
import random
import numpy as np

In [418]:
# Read the dataset
f = open("data/girls.txt", "r")
girls = []
for girl in f:
    girls.append(girl[:-1].lower())

f = open("data/boys.txt", "r")
boys = []
for boy in f:
    boys.append(boy[:-1].lower())

print(len(boys), len(girls))
print(girls)
print(boys)

222 280
['aaradhya', 'adah', 'adhira', 'alisha', 'amoli', 'anaisha', 'ananya', 'anika', 'anushka', 'asmee', 'avni', 'carina', 'chara', 'drishti', 'ela', 'eshika', 'geetika', 'gulika', 'hiya', 'hiral', 'ira', 'ishana', 'ishita', 'jeevika', 'kaia', 'kashvi', 'keya', 'kimaya', 'krisha', 'larisa', 'laasya', 'mahika', 'mayra', 'mehar', 'mirai', 'mishka', 'naitee', 'navya', 'nyra', 'nehrika', 'neysa', 'pavati', 'prisha', 'ryka', 'rebecca', 'saanvi', 'sahana', 'sai', 'saisha', 'saira', 'saloni', 'shanaya', 'shrishti', 'sneha', 'turvi', 'taahira', 'taara', 'tanvi', 'viti', 'zara', 'aagya', 'aaina', 'aas', 'akaljeet', 'amanroop', 'anika', 'birva', 'bisanpreet', 'charanpreet', 'dilreet', 'ekkam', 'faal', 'gurleen', 'gurmeet', 'heer', 'harleen', 'harveen', 'ikamroop', 'isha', 'ishmeet', 'katiya', 'mehr', 'nihaara', 'paakhi', 'parminder', 'simrat', 'sukhdeep', 'sukhleen', 'shirina', 'tavleen', 'ami', 'askini', 'anvi', 'bandhini', 'bansari', 'charmi', 'chavi', 'charul', 'drisna', 'dhara', 'dhruvi',

In [419]:
# Convert each name to feature vector
X = []
y = []

# Considering all bigrams and bias term
num_feats = 703    # 26^2 bigrams + 1 bias term = 703 features

for girl in girls:
  vec = np.zeros(num_feats)
  vec[num_feats-1] = 1   # Initialize bias term as 1

  # Consider all letters
  for letter in girl:
  # Skipping whitespace and extra characters
    if (ord(letter) < 97 or ord(letter) > 122):
      continue
    vec[ord(letter) - 97] += 1

  # Consider all bigrams
  for i in range(len(girl)-1):
    bigram = girl[i:i+2]
    if (ord(bigram[0]) < 97 or ord(bigram[0]) > 122 or ord(bigram[0]) < 97 or ord(bigram[1]) > 122):
      continue
    vec[26 + (ord(bigram[0]) - 97)*26 + (ord(bigram[1]) - 97)] += 1

  vec = vec.reshape(-1, 1)
  X.append(vec)
  y.append(-1)

for boy in boys:
  vec = np.zeros(num_feats)
  vec[num_feats-1] = 1

  # Consider all letters
  for letter in boy:
  # Skipping whitespace and extra characters
    if (ord(letter) < 97 or ord(letter) > 122):
      continue
    vec[ord(letter) - 97] += 1
  
  # Consider all bigrams
  for i in range(len(boy)-1):
    bigram = boy[i:i+2]
    if (ord(bigram[0]) < 97 or ord(bigram[0]) > 122 or ord(bigram[0]) < 97 or ord(bigram[1]) > 122):
      continue
    vec[26 + (ord(bigram[0]) - 97)*26 + (ord(bigram[1]) - 97)] += 1

  vec = vec.reshape(-1, 1)
  X.append(vec)
  y.append(1)

# Printing sample feature vector and label
print(X[0].shape)
print(y[0])

(703, 1)
-1


In [420]:
# Split training and test data

test_size = 0.1

data = list(zip(X, y))
random.shuffle(data)

split_index = int(len(data) * (1 - test_size))
train_data = data[:split_index]
test_data = data[split_index:]

X_train, y_train = zip(*train_data)
X_test, y_test = zip(*test_data)

X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train), np.array(y_test)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print(f'Number of training examples: {len(X_train)}')
print(f'Number of test examples: {len(X_test)}')

Number of training examples: 451
Number of test examples: 51


In [421]:
# Defining weights for perceptron
W = np.zeros((num_feats, 1))

In [422]:
def fit(X, y, W):
    m = X.shape[0]

    num_iterations = 0

    # Continue till labels are classified
    while True:
        miss = 0
        for j in range(m):
            Z = y[j]*(np.dot(W.T, X[j]))
            if (Z[0][0] <= 0):
                W = W + y[j]*X[j]
                miss += 1
        
        if (num_iterations % 100 == 0):
            print(f"Iteration {num_iterations}: {miss} missclassifications")
            
        num_iterations += 1

        if (miss == 0):
            print(f"Iteration {num_iterations}: Converged!")
            break
        
    return W

In [423]:
W = fit(X_train, y_train, W)

Iteration 0: 194 missclassifications
Iteration 100: 29 missclassifications
Iteration 200: 8 missclassifications
Iteration 202: Converged!


In [424]:
def get_accuracy(X, y, W):

    m = X.shape[0]
    print(f"Total number of test samples: {m}")

    miss = 0
    for j in range(m):
        Z = y[j]*(np.dot(W.T, X[j]))
        if (Z[0][0] <= 0):
            miss += 1
    
    print(f"Number of missclassified samples: {miss}")

    return (m-miss)/m

accuracy = get_accuracy(X_test, y_test, W)
print(f"Accuracy on test set: {accuracy}")
        

Total number of test samples: 51
Number of missclassified samples: 12
Accuracy on test set: 0.7647058823529411


In [427]:
def predict(name):
    # Convert name to feature vector

    name = name.lower()
    
    # Considering all bigrams and bias term
    num_feats = 703    # 26^2 bigrams + 1 bias term = 677 features

    vec = np.zeros(num_feats)
    vec[num_feats-1] = 1   # Initialize bias term as 1

    # Consider all letters
    for letter in name:
    # Skipping whitespace and extra characters
        if (ord(letter) < 97 or ord(letter) > 122):
            continue
        vec[ord(letter) - 97] += 1

    # Consider all bigrams
    for i in range(len(name)-1):
        bigram = name[i:i+2]
        if (ord(bigram[0]) < 97 or ord(bigram[0]) > 122 or ord(bigram[0]) < 97 or ord(bigram[1]) > 122):
            continue
        vec[26 + (ord(bigram[0]) - 97)*26 + (ord(bigram[1]) - 97)] += 1

    vec = vec.reshape(-1, 1)
    
    z = np.dot(W.T, vec)

    if z > 0:
        print("I am sure " + name + " is a boy.")
    elif z < 0:
        print("I am sure " + name + " is a girl.")
    else:
        print("I am not sure if " + name + " is a boy or a girl.")

In [428]:
# Testing with our own example
predict("Chandan")
predict("Chandanbala")

I am sure chandan is a boy.
I am sure chandanbala is a girl.
